In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import os
from model_darknet19 import darknet
from decode import decode
from utils import preprocess_image, postprocess, draw_detection, generate_colors
from config import anchors, class_names
from matplotlib.pyplot import imshow

In [2]:
model_path = os.path.join('yolo2_model','yolo2_coco.ckpt')

In [3]:
image_name ='car3.jpg'
image_file = os.path.join('images',image_name)
image_detection = os.path.join('images',"dect_car3.jpg")

image = cv2.imread(image_file)
image_shape = image.shape[:2]


In [4]:
input_size = (416,416)

image_cp = preprocess_image(image)
tf_image = tf.placeholder(tf.float32,[1,input_size[0],input_size[1],3])
model_output = darknet(tf_image)

output_sizes = input_size[0]//32, input_size[1]//32 # 特征图尺寸是图片下采样32倍
output_decoded = decode(model_output=model_output,output_sizes=output_sizes, num_class=len(class_names),anchors=anchors)

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess,model_path)
    
    bboxes,obj_probs,class_probs = sess.run(output_decoded,feed_dict={tf_image:image_cp})

INFO:tensorflow:Restoring parameters from yolo2_model\yolo2_coco.ckpt


In [6]:
bboxes,scores,class_max_index = postprocess(bboxes,obj_probs,class_probs,image_shape=image_shape)

colors = generate_colors(class_names)
img_detection = draw_detection(image, bboxes, scores, class_max_index, class_names, colors)


In [7]:
cv2.imwrite(image_detection, img_detection)
cv2.imshow("detection_results", img_detection)
cv2.waitKey(0)

-1